In [1]:
import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import scala.collection.mutable.ArrayBuffer
import java.util.Properties
import java.io.ByteArrayInputStream;
import scala.collection.JavaConversions._

import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}

import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1606995829709)
SparkSession available as 'spark'


import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import scala.collection.mutable.ArrayBuffer
import java.util.Properties
import java.io.ByteArrayInputStream
import scala.collection.JavaConversions._
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
import spark.implicits._


## 데이터 불러오기

In [2]:
sc

res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@7527a9b1


In [3]:
val plainText = sc.textFile("Data/proj/T5.csv")

plainText: org.apache.spark.rdd.RDD[String] = Data/proj/T5.csv MapPartitionsRDD[1] at textFile at <console>:41


In [4]:
val stopWords = scala.io.Source.fromFile("Data/practice6/stopwords.txt").getLines().toSet

stopWords: scala.collection.immutable.Set[String] = Set(down, it's, ourselves, that's, for, method, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, methods, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, t...

In [5]:
def plainTextToLemmas(text: String, stopWords: Set[String], pipeline:StanfordCoreNLP): Seq[String] = {
  val doc = new Annotation(text)
  pipeline.annotate(doc)
  val lemmas = new ArrayBuffer[String]()
  val sentences = doc.get(classOf[SentencesAnnotation])
  for (sentence <- sentences; token <- sentence.get(classOf[TokensAnnotation])) {
    val lemma = token.get(classOf[LemmaAnnotation])
    if (lemma.length > 2 && !stopWords.contains(lemma)) {
      lemmas += lemma.toLowerCase
    }
  }
  lemmas
}

val lemmatized = plainText.mapPartitions(strings => {
  val props = new Properties()
  props.put("annotators", "tokenize, ssplit, pos, lemma")
  val pipeline = new StanfordCoreNLP(props)
  strings.map(string => plainTextToLemmas(string, stopWords, pipeline))
})
//lemmatized.foreach(println)

plainTextToLemmas: (text: String, stopWords: Set[String], pipeline: edu.stanford.nlp.pipeline.StanfordCoreNLP)Seq[String]
lemmatized: org.apache.spark.rdd.RDD[Seq[String]] = MapPartitionsRDD[2] at mapPartitions at <console>:58


## word2vec

In [7]:
import org.apache.spark.mllib.feature.{Word2Vec, Word2VecModel}

val word2vec = new Word2Vec()
val model = word2vec.fit(lemmatized)
val synonyms = model.findSynonyms("blockchain", 100)

for((synonym, cosineSimilarity) <- synonyms) {
  println(s"$synonym $cosineSimilarity")
}

// Save and load model
model.save(sc, "myModelPath")
val sameModel = Word2VecModel.load(sc, "myModelPath")

consensus 0.21461164951324463
chain 0.2114528864622116
base 0.20180371403694153
management 0.187967449426651
key 0.17615555226802826
state 0.17279267311096191
distribute 0.16372162103652954
perform 0.1537478119134903
share 0.14828689396381378
peer 0.14213550090789795
integrity 0.14040042459964752
authentication 0.13889484107494354
efficient 0.13813062012195587
trust 0.13649748265743256
validation 0.1361583024263382
software 0.12799331545829773
network 0.12795385718345642
database 0.10710969567298889
access 0.10526232421398163
paired 0.10280188918113708
user 0.09632086753845215
program 0.09115133434534073
confidential 0.08733682334423065
node 0.08522427082061768
frame 0.07410840690135956
method 0.06495237350463867
wireless 0.061737559735774994
ledger 0.0584103949368
application 0.05811193585395813
multiple 0.05467619374394417
content 0.05329390615224838
systems 0.04808741435408592
media 0.044690463691949844
public 0.043381042778491974
medium 0.04289785772562027
identity 0.03928028047084

org.apache.hadoop.mapred.FileAlreadyExistsException:  Output directory file:/root/workspace/myModelPath/metadata already exists